In [ ]:
#bibs
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import cv2
import tensorflow as tf
import warnings
import pydot
import gc

from tqdm import tqdm

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers.core import Dropout, Lambda
from tensorflow.python.keras.layers.convolutional import Conv2D, Conv2DTranspose
from tensorflow.python.keras.layers.pooling import MaxPooling2D
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.vis_utils import plot_model

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

warnings.filterwarnings("ignore")

In [ ]:
#data paths
data_dir = os.path.join(os.getcwd(), 'data')

img_dir = os.path.join(data_dir, 'images_test') #hfusg images of atopic dermatitis
mask_dir = os.path.join(data_dir, 'masks_test') #masks of SLEB layer

In [ ]:
#data preparation
X, y = [], []
IMG_WIDTH, IMG_HEIGHT = 256, 256

#looping through image files
for file in tqdm(os.listdir(img_dir)):
    
    fullpath = os.path.join(img_dir, file)
    img = cv2.imread(fullpath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #resize images to the size of network input
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    X.append(img)


#looping through mask files
for file in tqdm(os.listdir(mask_dir)):
    
    fullpath = os.path.join(mask_dir, file)
    #getting into .mat file from matlab, where masks are storaged
    mat_file =  sio.loadmat(fullpath)
    mat_e= mat_file['E']
    mat_e= mat_e['e']
    mat_arrs = mat_e[0].ravel()
    arrs = mat_arrs[0]
    arr = arrs[0,0]['azs']
    #resize images to the size of network input
    arr = cv2.resize(arr, (IMG_WIDTH, IMG_HEIGHT))
    y.append(arr)

print(arr.shape)
print(img.shape)

In [ ]:
#correction of data - unifying images and masks dimensions
X, y = np.array(X), np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
#sanity check - image and mask
plt.figure(figsize=(10,10))
plt.imshow(X[1], cmap='gray')
plt.axis('off')

plt.figure(figsize=(10,10))
plt.imshow(y[1], cmap='gray')
plt.axis('off')

x_length = len(X)
y = y.reshape((x_length,256,256,1))

In [ ]:
#data augmentation
def data_aug(X_train,X_test,y_train,y_test,train_batch_size,test_batch_size):
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0,
        height_shift_range=0,
        rescale=1.0/255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    train_batch = train_datagen.flow(X_train,y_train,batch_size=train_batch_size)
    test_batch = test_datagen.flow(X_test,y_test,batch_size=test_batch_size)
    return (train_batch,test_batch)

In [ ]:
#metric function Intersection over Union IoU (Jaccard Index)
smooth = 1.
def mean_iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

In [ ]:
#convolutional neural network model (u-net architecture)
def create_unet(img_height, img_width, channels): 
    inputs = Input((img_height, img_width,  channels))

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    #output is a mask, size of 256x256
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
    model.summary()
    return model

In [ ]:
#callbacks setting
def callbacks():
    
    callback = []
    tb = TensorBoard(log_dir='logs', histogram_freq=0,
                          write_graph=True, write_images=False)
    callback.append(tb)
    #adding different callbacks
    
    #checking learning progress
    checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
    callback.append(checkpoint)
    #early stoppage of epoch
    earlystop = EarlyStopping(monitor = "val_loss", #val_loss 
                              min_delta = 0, 
                              patience = 8,
                              mode="min",
                              verbose = 1,
                              restore_best_weights = True)
    callback.append(earlystop)

    #reducing learning rate value 
    reducelr = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.2,
                                  patience=3, 
                                  min_lr=0.0001, 
                                  verbose=1)
    callback.append(reducelr)
    
    return callback

In [ ]:
#setting cross-validation and learning model
kfold = KFold(n_splits=4, shuffle=True, random_state=42)
cvscores = []
fold_number = 1
train_X = X
train_y = y
for train_index, val_index in kfold.split(train_X):
    gc.collect()
    K.clear_session()
    print ('Fold: ',fold_number)
    X_train, X_val = train_X[train_index], train_X[val_index]
    y_train, y_val = train_y[train_index], train_y[val_index]
    
    #data augmentation
    batch_size = 5
    train_batch, val_batch = data_aug(X_train,X_val,y_train,y_val, batch_size, batch_size)
    model_name = 'unet_fold_'+str(fold_number)+'.h5'
    callback = callbacks()
    
    #create unet model
    model = create_unet(IMG_HEIGHT,IMG_WIDTH,3)
    
  
    epochs = 25 
    model.fit_generator(train_batch, validation_data=val_batch, epochs=epochs, 
                        validation_steps= X_val.shape[0] // batch_size, 
                        steps_per_epoch= X_train.shape[0] // batch_size, 
                        callbacks=callback)
    
    #save model from the current fold
    model.save(model_name)
    
    #evaluate the model
    scores = model.evaluate(X_val, y_val, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)   
    
    fold_number = fold_number + 1

print("%s: %.2f%%" % ("Mean Accuracy: ",np.mean(cvscores)))
print("%s: %.2f%%" % ("Standard Deviation: +/-", np.std(cvscores)))

In [ ]:
#%tensorboard --logdir logs

In [ ]:
#ensembling models created by cross-validation
def ensemble(models, model_input):
    
    Models_output=[model(model_input) for model in models]
    avg = keras.layers.average(Models_output)
    
    modelEnsemble = Model(inputs=model_input, outputs=avg, name='ensemble')
    modelEnsemble.summary()
    modelEnsemble.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mean_iou'])
    return modelEnsemble

In [ ]:
model_fold_1 = create_unet(IMG_HEIGHT,IMG_WIDTH,3)
model_fold_2 = create_unet(IMG_HEIGHT,IMG_WIDTH,3)
model_fold_3 = create_unet(IMG_HEIGHT,IMG_WIDTH,3)
model_fold_4 = create_unet(IMG_HEIGHT,IMG_WIDTH,3)

models = []

#load weights from each model
model_fold_1.load_weights('unet_fold_1.h5')
model_fold_1.name = 'model_1'
models.append(model_1)

model_fold_2.load_weights('unet_fold_2.h5')
model_fold_2.name = 'model_2'
models.append(model_2)

model_fold_3.load_weights('unet_fold_3.h5')
model_fold_3.name = 'model_3'
models.append(model_3)

model_fold_4.load_weights('unet_fold_4.h5')
model_fold_4.name = 'model_4'
models.append(model_4)

model_input = Input(shape=models[0].input_shape[1:])
ensemble_model = ensemble(models, model_input)

In [ ]:
#evaluate ensembled model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
scores = ensemble_model.evaluate(X_val, y_val, verbose=0)
print("%s: %.2f%%" % (ensemble_model.metrics_names[1], scores[1]*100))

model_name = 'unet_ensembled.h5'
ensemble_model.save(model_name)

#plot network architecture
plot_model(ensemble_model, show_shapes=True, show_layer_names=False)
img = cv2.imread('model.png',1)
plt.figure(figsize=(30,15))
plt.imshow(img)


In [ ]:
#predict data
probability = ensemble_model.predict(X,batch_size=None,steps=1)
print(probability)

In [ ]:
#network prediction of SLEB layer
x = x.reshape(1,256,256,3)
pred = model.predict(x)
pred = pred.reshape(256,256)
print(np.unique(pred))

for w in range(0,256):
    for h  in range(0,256):
        
        if pred[w,h] > 0.9: #threshold of propability matrix, range [0,1].
                            #1 is 100% sureness of network pixel belongs to SLEB layer
            pred[w,h] = 1
        else:
            pred[w,h] = 0
plt.figure(figsize=(10,10))
plt.imshow(pred, cmap='gray')
plt.axis('off')
